In [1]:
!pip install deepstack-python


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
!pip install deepstack-sdk



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import zipfile
import os

zip_file_path = 'dataset.zip' 
extract_target_dir = './Anomaly_Dataset'
target_folders = [
    'Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1',
    'Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2',
    'Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1'
]

os.makedirs(extract_target_dir, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    all_files = zip_ref.namelist()

    for file in all_files:
        if any(file.startswith(folder) for folder in target_folders):
            zip_ref.extract(file, extract_target_dir)

print("Extraction completed for the specified folders.")

extracted_folders = [
    'Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1',
    'Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2',
    'Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1'
]

for folder in extracted_folders:
    folder_path = os.path.join('.', folder)
    if os.path.exists(folder_path):
        print(f"Folder {folder_path} exists and has been extracted.")
    else:
        print(f"Folder {folder_path} does not exist. Extraction might have failed.")

if os.path.exists(zip_file_path):
    os.remove(zip_file_path)
    print(f"Removed the zip file: {zip_file_path}")
else:
    print(f"The zip file {zip_file_path} was not found.")


Extraction completed for the specified folders.
Folder .\Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-1 does not exist. Extraction might have failed.
Folder .\Anomaly_Dataset/Anomaly_Videos/Anomaly-Videos-Part-2 does not exist. Extraction might have failed.
Folder .\Anomaly_Dataset/Anomaly_Videos/Normal-Videos-Part-1 does not exist. Extraction might have failed.
Removed the zip file: dataset.zip


In [5]:
import cv2
from deepstack_sdk import ServerConfig, Detection
import os
import time
from urllib3.exceptions import ProtocolError
from socket import gaierror

# Configure the DeepStack server
config = ServerConfig("http://localhost:5001")
detection = Detection(config)

def frames_already_extracted(video_name, output_folder):
    for root, _, files in os.walk(output_folder):
        for file in files:
            if file.startswith(video_name):
                return True
    return False

def saveFrames(video_path, output_folder):
    video_name = os.path.basename(video_path).split('.')[0]

    # Check if frames have already been extracted for this video
    if frames_already_extracted(video_name, output_folder):
        print(f"Frames already extracted for video: {video_name}. Skipping.")
        return

    cap = cv2.VideoCapture(video_path)

    frame_interval = 30
    frame_skip_interval = 10
    frame_count = 1
    saved_frame_count = 0

    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return

    retry_attempts = 5
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            try:
                ret, buffer = cv2.imencode('.jpg', frame)
                if not ret:
                    print(f"Error encoding frame at {frame_count}")
                    continue

                response = detection.detectObject(buffer.tobytes())

                for obj in response:
                    if obj.label == "person" and obj.confidence > 0.70:
                        if frame_count % (frame_interval * frame_skip_interval) == 0:
                            output_path = os.path.join(output_folder, f'{video_name}_fr{str(saved_frame_count)}.jpg')
                            cv2.imwrite(output_path, frame)
                            saved_frame_count += 1
                            break
            except ProtocolError as e:
                print(f"ProtocolError encountered: {e}")
                print("Retrying after a short delay...")
                time.sleep(5)
            except gaierror as e:
                print(f"gaierror encountered: {e}")
                print("Retrying after a short delay...")
                time.sleep(5)
            except Exception as e:
                if '429' in str(e):
                    for i in range(retry_attempts):
                        print(f"Error 429 encountered: {e}. Retrying attempt {i+1}/{retry_attempts} after {2 ** i} seconds...")
                        time.sleep(2 ** i)  # Exponential backoff
                        try:
                            response = detection.detectObject(buffer.tobytes())
                            break
                        except Exception as retry_e:
                            if i == retry_attempts - 1:
                                print(f"Max retry attempts reached. Skipping frame {frame_count} due to error 429.")
                            else:
                                continue
                else:
                    print(f"An error occurred: {e}")

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

    # Print message after processing each video
    print(f"Completed processing video: {video_name}, saved {saved_frame_count} frames.")

def process_videos_in_directory(directory, output_folder):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.mp4'):
                video_path = os.path.join(root, file)
                saveFrames(video_path, output_folder)


base_path = r'C:\Users\swadh\OneDrive\Desktop\crimedetection\Anomaly_Dataset\Anomaly_Dataset\Anomaly_Videos'

print("Listing contents of the base path:")
try:
    base_contents = os.listdir(base_path)
    print(base_contents)
except Exception as e:
    print(f"Error listing contents of base path: {e}")

video_dirs = [
    # os.path.join(base_path, 'Anomaly-Videos-Part-1/Abuse'),
    # os.path.join(base_path, 'Anomaly-Videos-Part-1/Arrest'),
    # os.path.join(base_path, 'Anomaly-Videos-Part-1/Assault'),
    os.path.join(base_path, 'Anomaly-Videos-Part-2/Fighting'),
    os.path.join(base_path, 'Anomaly-Videos-Part-2/Burglary')
]

output_dir = './Frames_for_Crime'
os.makedirs(output_dir, exist_ok=True)

for video_dir in video_dirs:
    if os.path.exists(video_dir):
        print(f"Processing directory: {video_dir}")
        process_videos_in_directory(video_dir, output_dir)
    else:
        print(f"Error: Directory {video_dir} does not exist. Skipping.")

print("Frame extraction completed.")


Listing contents of the base path:
['Anomaly-Videos-Part-1', 'Anomaly-Videos-Part-2', 'Normal-Videos-Part-1']
Processing directory: C:\Users\swadh\OneDrive\Desktop\crimedetection\Anomaly_Dataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-2/Fighting
Completed processing video: Fighting002_x264, saved 0 frames.
Frames already extracted for video: Fighting003_x264. Skipping.
Frames already extracted for video: Fighting004_x264. Skipping.
Completed processing video: Fighting005_x264, saved 0 frames.
Frames already extracted for video: Fighting006_x264. Skipping.
Frames already extracted for video: Fighting007_x264. Skipping.
Frames already extracted for video: Fighting008_x264. Skipping.
Completed processing video: Fighting009_x264, saved 0 frames.
Frames already extracted for video: Fighting010_x264. Skipping.
Completed processing video: Fighting011_x264, saved 25 frames.
Completed processing video: Fighting012_x264, saved 0 frames.
Completed processing video: Fighting013_x264, sa

In [10]:
import cv2
from deepstack_sdk import ServerConfig, Detection
import os
import time
from urllib3.exceptions import ProtocolError
from socket import gaierror

# Configure the DeepStack server
config = ServerConfig("http://localhost:5001")
detection = Detection(config)

def frames_already_extracted(video_name, output_folder):
    for root, _, files in os.walk(output_folder):
        for file in files:
            if file.startswith(video_name):
                return True
    return False

def saveFrames(video_path, output_folder):
    video_name = os.path.basename(video_path).split('.')[0]

    # Check if frames have already been extracted for this video
    if frames_already_extracted(video_name, output_folder):
        print(f"Frames already extracted for video: {video_name}. Skipping.")
        return

    cap = cv2.VideoCapture(video_path)

    frame_interval = 30
    frame_skip_interval = 10
    frame_count = 1
    saved_frame_count = 0

    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return

    retry_attempts = 5
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            try:
                ret, buffer = cv2.imencode('.jpg', frame)
                if not ret:
                    print(f"Error encoding frame at {frame_count}")
                    continue

                response = detection.detectObject(buffer.tobytes())

                for obj in response:
                    if obj.label == "person" and obj.confidence > 0.70:
                        if frame_count % (frame_interval * frame_skip_interval) == 0:
                            output_path = os.path.join(output_folder, f'{video_name}_fr{str(saved_frame_count)}.jpg')
                            cv2.imwrite(output_path, frame)
                            saved_frame_count += 1
                            break
            except ProtocolError as e:
                print(f"ProtocolError encountered: {e}")
                print("Retrying after a short delay...")
                time.sleep(5)
            except gaierror as e:
                print(f"gaierror encountered: {e}")
                print("Retrying after a short delay...")
                time.sleep(5)
            except Exception as e:
                if '429' in str(e):
                    for i in range(retry_attempts):
                        print(f"Error 429 encountered: {e}. Retrying attempt {i+1}/{retry_attempts} after {2 ** i} seconds...")
                        time.sleep(2 ** i)  # Exponential backoff
                        try:
                            response = detection.detectObject(buffer.tobytes())
                            break
                        except Exception as retry_e:
                            if i == retry_attempts - 1:
                                print(f"Max retry attempts reached. Skipping frame {frame_count} due to error 429.")
                            else:
                                continue
                else:
                    print(f"An error occurred: {e}")

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

    # Print message after processing each video
    print(f"Completed processing video: {video_name}, saved {saved_frame_count} frames.")

def process_videos_in_directory(directory, output_folder):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.mp4'):
                video_path = os.path.join(root, file)
                saveFrames(video_path, output_folder)


base_path = r'C:\Users\swadh\OneDrive\Desktop\crimedetection\Anomaly_Dataset\Anomaly_Dataset\Anomaly_Videos'

print("Listing contents of the base path:")
try:
    base_contents = os.listdir(base_path)
    print(base_contents)
except Exception as e:
    print(f"Error listing contents of base path: {e}")

video_dirs = [
     os.path.join(base_path, 'Anomaly-Videos-Part-1/Abuse'),
     os.path.join(base_path, 'Anomaly-Videos-Part-1/Arrest'),
     os.path.join(base_path, 'Anomaly-Videos-Part-1/Assault'),
    #os.path.join(base_path, 'Anomaly-Videos-Part-2/Fighting'),
    #os.path.join(base_path, 'Anomaly-Videos-Part-2/Burglary')
]

output_dir = './Frames_for_Crime'
os.makedirs(output_dir, exist_ok=True)

for video_dir in video_dirs:
    if os.path.exists(video_dir):
        print(f"Processing directory: {video_dir}")
        process_videos_in_directory(video_dir, output_dir)
    else:
        print(f"Error: Directory {video_dir} does not exist. Skipping.")

print("Frame extraction completed.")


Listing contents of the base path:
['Anomaly-Videos-Part-1', 'Anomaly-Videos-Part-2', 'Normal-Videos-Part-1']
Processing directory: C:\Users\swadh\OneDrive\Desktop\crimedetection\Anomaly_Dataset\Anomaly_Dataset\Anomaly_Videos\Anomaly-Videos-Part-1/Abuse
Completed processing video: Abuse001_x264, saved 2 frames.
Completed processing video: Abuse002_x264, saved 0 frames.
Completed processing video: Abuse003_x264, saved 2 frames.
Completed processing video: Abuse004_x264, saved 16 frames.
Completed processing video: Abuse005_x264, saved 2 frames.
Completed processing video: Abuse006_x264, saved 2 frames.
Completed processing video: Abuse007_x264, saved 3 frames.
Completed processing video: Abuse008_x264, saved 14 frames.
Completed processing video: Abuse009_x264, saved 2 frames.
Completed processing video: Abuse010_x264, saved 2 frames.
Completed processing video: Abuse011_x264, saved 1 frames.
Completed processing video: Abuse012_x264, saved 11 frames.
Completed processing video: Abuse01

In [9]:
import cv2
from deepstack_sdk import ServerConfig, Detection
import os
import time
from urllib3.exceptions import ProtocolError
from socket import gaierror

config = ServerConfig("http://localhost:5001")
detection = Detection(config)

def frames_already_extracted(video_name, output_folder):
    for root, _, files in os.walk(output_folder):
        for file in files:
            if file.startswith(video_name):
                return True
    return False

def saveFrames(video_path, output_folder):
    video_name = os.path.basename(video_path).split('.')[0]

    # Check if frames have already been extracted for this video
    if frames_already_extracted(video_name, output_folder):
        print(f"Frames already extracted for video: {video_name}. Skipping.")
        return

    cap = cv2.VideoCapture(video_path)

    frame_interval = 30
    frame_skip_interval = 10
    frame_count = 1
    saved_frame_count = 0

    if not cap.isOpened():
        print(f"Error: Unable to open video file {video_path}")
        return

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            try:
                ret, buffer = cv2.imencode('.jpg', frame)
                if not ret:
                    print(f"Error encoding frame at {frame_count}")
                    continue

                response = detection.detectObject(buffer.tobytes())

                for obj in response:
                    if obj.label == "person" and obj.confidence > 0.70:
                        if frame_count % (frame_interval * frame_skip_interval) == 0:
                            output_path = os.path.join(output_folder, f'{video_name}_fr{str(saved_frame_count)}.jpg')
                            cv2.imwrite(output_path, frame)
                            saved_frame_count += 1
                            break
            except ProtocolError as e:
                print(f"ProtocolError encountered: {e}")
                print("Retrying after a short delay...")
                time.sleep(5)
            except gaierror as e:
                print(f"gaierror encountered: {e}")
                print("Retrying after a short delay...")
                time.sleep(5)
            except Exception as e:
                print(f"An error occurred: {e}")

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

    # Print message after processing each video
    print(f"Completed processing video: {video_name}, saved {saved_frame_count} frames.")

def process_videos_in_directory(directory, output_folder):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.mp4'):
                video_path = os.path.join(root, file)
                saveFrames(video_path, output_folder)

base_path = r'C:\Users\swadh\OneDrive\Desktop\crimedetection\Anomaly_Dataset\Anomaly_Dataset\Anomaly_Videos'

print("Listing contents of the base path:")
print(os.listdir(base_path))

video_dirs = [
    os.path.join(base_path, 'Normal-Videos-Part-1')
]

output_dir = './Frames_for_Normal'
os.makedirs(output_dir, exist_ok=True)

for video_dir in video_dirs:
    if os.path.exists(video_dir):
        process_videos_in_directory(video_dir, output_dir)
    else:
        print(f"Error: Directory {video_dir} does not exist. Skipping.")

print("Frame extraction completed.")


Listing contents of the base path:
['Anomaly-Videos-Part-1', 'Anomaly-Videos-Part-2', 'Normal-Videos-Part-1']
Completed processing video: Normal_Videos_003_x264, saved 0 frames.
Completed processing video: Normal_Videos_006_x264, saved 0 frames.
Completed processing video: Normal_Videos_010_x264, saved 3 frames.
Completed processing video: Normal_Videos_014_x264, saved 1 frames.
Completed processing video: Normal_Videos_015_x264, saved 0 frames.
Completed processing video: Normal_Videos_018_x264, saved 2 frames.
Completed processing video: Normal_Videos_019_x264, saved 5 frames.
Completed processing video: Normal_Videos_024_x264, saved 3 frames.
Completed processing video: Normal_Videos_025_x264, saved 2 frames.
Completed processing video: Normal_Videos_027_x264, saved 7 frames.
Completed processing video: Normal_Videos_033_x264, saved 2 frames.
Completed processing video: Normal_Videos_034_x264, saved 1 frames.
Completed processing video: Normal_Videos_041_x264, saved 2 frames.
Comple

In [13]:
import os
import shutil


crime_frames_dir = r'C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime'


fighting_dir = os.path.join(crime_frames_dir, 'Fighting')
burglary_dir = os.path.join(crime_frames_dir, 'Burglary')
abuse_dir = os.path.join(crime_frames_dir, 'Abuse')
arrest_dir = os.path.join(crime_frames_dir, 'Arrest')
assault_dir = os.path.join(crime_frames_dir, 'Assault')


os.makedirs(fighting_dir, exist_ok=True)
os.makedirs(burglary_dir, exist_ok=True)
os.makedirs(arrest_dir, exist_ok=True)
os.makedirs(assault_dir, exist_ok=True)
os.makedirs(abuse_dir, exist_ok=True)


for filename in os.listdir(crime_frames_dir):
    if 'fighting' in filename.lower():
        shutil.move(os.path.join(crime_frames_dir, filename), fighting_dir)
    elif 'burglary' in filename.lower():
        shutil.move(os.path.join(crime_frames_dir, filename), burglary_dir)
    elif 'abuse' in filename.lower():
       shutil.move(os.path.join(crime_frames_dir, filename),abuse_dir)
    elif 'arrest' in filename.lower():
       shutil.move(os.path.join(crime_frames_dir, filename), arrest_dir)
    elif 'assault' in filename.lower():
       shutil.move(os.path.join(crime_frames_dir, filename), assault_dir)
   


print("Files have been organized into subfolders.")


Files have been organized into subfolders.


In [14]:
import os

def count_images_in_directory(directory):
    count = 0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
                count += 1
    return count

fighting_path = r'C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime\Fighting'
burglary_path=r'C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime\Burglary'
abuse_path=r'C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime\Abuse'
arrest_path=r'C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime\Arrest'
assault_path=r'C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime\Assault'
normal_path=r'C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Normal'

burglary_count = count_images_in_directory(burglary_path)
print(f"Number of images in {burglary_path}: {burglary_count}")
fighting_count = count_images_in_directory(fighting_path)
print(f"Number of images in {fighting_path}: {fighting_count}")
Abuse_count = count_images_in_directory(abuse_path)
print(f"Number of images in {abuse_path}: {Abuse_count}")
Arrest_count = count_images_in_directory(arrest_path)
print(f"Number of images in {arrest_path}: {Arrest_count}")
Assault_count = count_images_in_directory(assault_path)
print(f"Number of images in {assault_path}: {Assault_count}")

normal_count = count_images_in_directory(normal_path)
print(f"Number of images in {normal_path}: {normal_count}")


Number of images in C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime\Burglary: 313
Number of images in C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime\Fighting: 460
Number of images in C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime\Abuse: 256
Number of images in C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime\Arrest: 430
Number of images in C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime\Assault: 197
Number of images in C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Normal: 1130


In [15]:
import os
import shutil
import random

def create_test_dataset(source_folder, dest_folder, test_ratio=0.2):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    for subdir in os.listdir(source_folder):
        subdir_path = os.path.join(source_folder, subdir)
        if os.path.isdir(subdir_path):
            
            dest_subdir_path = os.path.join(dest_folder, subdir)
            os.makedirs(dest_subdir_path, exist_ok=True)

            files = [f for f in os.listdir(subdir_path) if os.path.isfile(os.path.join(subdir_path, f))]

            num_files = len(files)
            num_test_files = int(num_files * test_ratio)

            test_files = random.sample(files, num_test_files)
            

            for file in test_files:
                shutil.move(os.path.join(subdir_path, file), os.path.join(dest_subdir_path, file))

            print(f"Moved {num_test_files} files from {subdir_path} to {dest_subdir_path}")


Frames_for_Crime = r'C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime'

# Define destination folders for the test dataset
test_frames_for_crime = 'path_to_test_frames_for_crime'

# Create test datasets for crime and non-crime images, maintaining the same subfolder structure
create_test_dataset(Frames_for_Crime, test_frames_for_crime, test_ratio=0.2)



Moved 51 files from C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime\Abuse to path_to_test_frames_for_crime\Abuse
Moved 86 files from C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime\Arrest to path_to_test_frames_for_crime\Arrest
Moved 39 files from C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime\Assault to path_to_test_frames_for_crime\Assault
Moved 62 files from C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime\Burglary to path_to_test_frames_for_crime\Burglary
Moved 92 files from C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Crime\Fighting to path_to_test_frames_for_crime\Fighting


In [16]:
import os
import shutil
import random

def create_test_dataset(source_folder, dest_folder, test_ratio=0.2):
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)

    valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
    files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f)) and f.lower().endswith(valid_extensions)]

    num_files = len(files)
    num_test_files = int(num_files * test_ratio)

    test_files = random.sample(files, num_test_files)

    for file in test_files:
        shutil.move(os.path.join(source_folder, file), os.path.join(dest_folder, file))

    print(f"Moved {num_test_files} files from {source_folder} to {dest_folder}")

Frames_for_Normal = r'C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Normal'
test_frames_for_normal = r'C:\Users\swadh\OneDrive\Desktop\crimedetection\test_frames_for_normal'

create_test_dataset(Frames_for_Normal, test_frames_for_normal, test_ratio=0.2)


Moved 226 files from C:\Users\swadh\OneDrive\Desktop\crimedetection\Frames_for_Normal to C:\Users\swadh\OneDrive\Desktop\crimedetection\test_frames_for_normal
